In [ ]:
!nvidia-smi

In [ ]:
!pip install tensorflow==2.13.0 opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

In [ ]:
data_dir="dataset"

In [ ]:
classes=["bio","cardboard","e-waste","glass","metal","paper","plastics","trash"]

In [ ]:
os.listdir(os.path.join(data_dir,"cardboard"))[0:5]

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
img_ext=['jpeg','jpg','png']


In [ ]:
for img_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,img_class)):
        img_path=os.path.join(data_dir,img_class,image)
        try:
            img=cv2.imread(img_path)
            ext=imghdr.what(img_path)
            if ext not in img_ext:
                print("image is not in ext list {}".format(img_path))
                os.remove(img_path)
        except Exception as e:
                      print("issue with image{}".format(e))

In [ ]:
import numpy as np

In [ ]:
data=tf.keras.utils.image_dataset_from_directory(data_dir,batch_size=20)

In [ ]:

len(data)

In [ ]:
data=data.map(lambda x,y:(x/255,y))

In [ ]:
data_iterator=data.as_numpy_iterator()

In [ ]:
batch=data_iterator.next()

In [ ]:
len(batch)

In [ ]:
batch[0]

In [ ]:
batch[0].shape

In [ ]:
batch[1]

In [ ]:
fig,ax=plt.subplots(ncols=5,nrows=2,figsize=(20,10))
fig.subplots_adjust(hspace=0)
for idx,img in enumerate(batch[0][:10]):
                         ax[idx//5][idx%5].imshow(img)
                         ax[idx//5][idx%5].set_title(batch[1][idx])


In [ ]:
train_size=int(len(data)*0.7)
val_size=int(len(data)*0.2)+1
test_size=int(len(data)*0.1)+1

In [ ]:
train_data=data.take(train_size)
val_data=data.skip(train_size).take(val_size)
test_data=data.skip(train_size+val_size).take(test_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Dense,Flatten,BatchNormalization,Dropout

In [ ]:
!pip install keras-tuner

In [ ]:
model=Sequential()

In [ ]:
def build_model(hp):
    model=Sequential()
    model.add(Conv2D(filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),activation="relu",input_shape=(256,256,3)))

    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.05)))

    model.add(MaxPooling2D())
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=hp.Int('conv_3_filter',min_value=32,max_value=64,step=16),kernel_size=hp.Choice('conv_3_kernel',values=[3,5]),activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.03)))
    model.add(Dropout(0.1))
    model.add(Dense(8,activation="softmax"))
    model.compile(optimizer=tf.keras.optimizers.SGD(hp.Choice("learning_rate",values=[1e-2,1e-3])),loss="sparse_categorical_crossentropy",metrics=["accuracy"])
    return model

In [ ]:
from kerastuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters



In [ ]:
tunner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory="output",project_name="wasteClassifier")

In [ ]:
tunner_search.search(train_data,epochs=3,validation_data=val_data)

In [ ]:
model=tunner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
!pip install pydot pydotplus graphviz

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model)

In [ ]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir='logs')

In [ ]:
result=model.fit(train_data,validation_data=val_data,epochs=50,initial_epoch=3,callbacks=tensorboard_callback)

In [ ]:
result.history

In [ ]:
plt.plot(result.history['loss'], color='teal', label='loss')
plt.plot(result.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.plot(result.history['accuracy'], color='teal', label='accuracy')
plt.plot(result.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
for batch in test_data.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    yhat=np.array(list(map(lambda x: np.argmax(x),yhat)))



In [ ]:
model.evaluate(test_data)

In [ ]:
image=cv2.imread("download (1).jpg")

In [ ]:
img=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

In [ ]:
 plt.imshow(img)

In [ ]:
resize_img=tf.image.resize(img,(256,256))
plt.imshow(resize_img.numpy().astype(int))

In [ ]:
yhat=model.predict(np.expand_dims(resize_img/255,0))

In [ ]:
classes[np.argmax(yhat)]

In [ ]:
from keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
model_json = model.to_json()
with open("models/model.json", "w") as json_file:
    json_file.write(model_json)




In [ ]:
new_model = load_model(os.path.join('models','imageclassifier.h5'))

In [ ]:
yhat=new_model.predict(np.expand_dims(resize_img/255,0))

In [ ]:
classes[np.argmax(yhat)]